In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import redis
import pendulum
import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-logging')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--ignore-certificate-errors')
browser = webdriver.Chrome(chrome_options=chrome_options)

In [3]:
broker_url = 'redis://localhost:6379/0'
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
p4p = P4P(market, market['lid'], market['lpwd'],broker_url=broker_url, headless=False, browser=browser)

. storage markets.json
.\storage\markets.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!


In [ ]:
p4p.crawl = types.MethodType(crawl, p4p)

In [ ]:
p4p.save_crawling_result = types.MethodType(save_crawling_result, p4p)

In [ ]:
p4p.find_prices_and_sponsors = types.MethodType(find_prices_and_sponsors, p4p)

In [40]:
def find_sponsors(self, kws):
    url = 'https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&viewtype=L&CatId=&SearchText=' + re.sub(
        ' +', '+', kws)
    self.headers['Referer'] = url

    response = None
    while response is None:
        try:
            response = requests.get(url, headers=self.headers)
            break
        except Exception as e:
            traceback.print_exc()
            time.sleep(3)
            print('====================== retry in 3 seconds =================================')
            response = None
            continue

    top_sponsor = None
    sponsor_list = []

    result = re.search(r'_search_result_data =(.*)page.setPageData\(_search_result_data\)', response.text,
                       re.M | re.DOTALL)

    result = re.sub(',[\n\t ]*}', '}',result.group(1))
    result = re.sub('"" === "true"', '"\\"\\" === \\"true\\""', result)
    
    obj = json.loads(result)
    items = obj['normalList']
    for idx, item in enumerate(items):
        company = {}
        if (item['isBrandAd']):
            top_sponsor = company
        elif (item['isP4p']):
            sponsor_list.append(company)
        else:
            break

        company['years'] = item['supplierYear']
        company['name'] = item['supplierName']
        company['url'] = item['supplierHref']
        if 'record' in item:
            company['record'] = []
            if 'transaction' in item['record']:
                company['record'].append(item['record']['transaction']['num'])
                company['record'].append('Transactions(6 months)')
                company['record'].append(item['record']['transaction']['conducted'])
            if 'responseRate' in item['record']:
                company['record'].append('Response Rate')
                company['record'].append(item['record']['responseRate'])

    return {'top_sponsor': top_sponsor, 'sponsor_list': sponsor_list}
p4p.find_sponsors = types.MethodType(find_sponsors, p4p)

In [ ]:
p4p.browser.set_window_size(1920, 1200)

In [ ]:
p4p.browser.save_screenshot('screenie.png')

In [4]:
group = '关注词_2'
group = '直通车高消费词'

In [5]:
p4p.load_url()
p4p.switch_to_group(group)

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


3

In [ ]:
p4p.crawl(group=group)

In [6]:
p4p.set_sub_budget(85)

In [13]:
p4p.unset_sub_budget()

In [41]:
p4p.monitor(group=group)

Task Monitor start to run with group="直通车高消费词"
switch to group: 直通车高消费词
index: 0 3 > eyelash extension >  {

	"baseServer":"//www.alibaba.com/",
	"util":{
		"currentType":"product",
		"keyword":"eyelash\u0020extension",
		"aggregationName":"",
		"cateId":"",
		"cateName":"",
		"topicPosition":"6",
		"topicShow":true,
		"virtualExposurePageId":"a15703b9af7b41cf8b2f1f7e95275c37",
		            "num":"142,405"
			},
		"dmtrack":{
		"dmtrack_c":"industry_id=66010405|sn_cat_type=2|qrw_kw=eyelash+extension|p4pid=c4360662-6ae8-460d-872c-160900d83a25|s_abtest=01_M_R|se_pn=1|isAtmOnline=0|is_broadquery=0|sq_type=a_j|forecast_post_cate=66010405:0:255.0|is_f0_test=1|kw_type=0|is_chinasuppliers=0|isGalleryList=0|pvmi=f35b41ca3e554f859b5319dfc3412258|carry_flag=product|magellan_broadSn_offline=old_version|sek=eyelash+extension_eyelash+extent|semi=2|main_search_theme=new_version|is_onsite_check=0|is_gs_filt=0|proxy_ip=0|se_rst=142405|magellan_spu=new_version|ser=1001|bran_ab_test=0|is_waterfall=0|se

make selected >  >>>>>> successful end 
index: 1 3 > magnetic eyelashes >  {

	"baseServer":"//www.alibaba.com/",
	"util":{
		"currentType":"product",
		"keyword":"magnetic\u0020eyelashes",
		"aggregationName":"",
		"cateId":"",
		"cateName":"",
		"topicPosition":"5",
		"topicShow":true,
		"virtualExposurePageId":"c03bf1e125a3429faca841ae62ab688c",
		            "num":"25,427"
			},
		"dmtrack":{
		"dmtrack_c":"industry_id=66010405|sn_cat_type=1|qrw_kw=magnetic+eyelashes|p4pid=9b22e919-c466-4bcc-948c-bc0190c8bbcf|s_abtest=01_M_R|se_pn=1|isAtmOnline=0|is_broadquery=0|sq_type=a_j|forecast_post_cate=66010405:0:255.0|is_f0_test=1|kw_type=0|is_chinasuppliers=0|isGalleryList=0|pvmi=ddbc2fbd0f454be7b1093a7507f312ce|carry_flag=product|magellan_broadSn_offline=old_version|sek=magnetic+eyelashes_magnetic+eye+lashes|semi=2|main_search_theme=new_version|is_onsite_check=0|is_gs_filt=0|proxy_ip=0|se_rst=25427|magellan_spu=new_version|ser=1001|bran_ab_test=0|is_waterfall=0|set=3|change_viewtype=true|

make selected >  >>>>>> successful end 
index: 2 3 > mink eyelashes >  {

	"baseServer":"//www.alibaba.com/",
	"util":{
		"currentType":"product",
		"keyword":"mink\u0020eyelashes",
		"aggregationName":"",
		"cateId":"",
		"cateName":"",
		"topicPosition":"6",
		"topicShow":true,
		"virtualExposurePageId":"597e9dee53234685b8c683c52cf537f4",
		            "num":"206,043"
			},
		"dmtrack":{
		"dmtrack_c":"industry_id=66010405|sn_cat_type=2|qrw_kw=mink+eyelashes|p4pid=134e94c5-97b0-414c-9f4c-f01e2dd18c0e|s_abtest=01_M_R|se_pn=1|isAtmOnline=0|is_broadquery=0|sq_type=a_j_j|forecast_post_cate=66010405:0:255.0|is_f0_test=1|kw_type=0|is_chinasuppliers=0|isGalleryList=0|pvmi=26a1f27516904856ac924ce50d39f45f|carry_flag=product|magellan_broadSn_offline=old_version|sek=mink+eyelashes_fur+eyelashes_mink+eye+lashes|semi=2|main_search_theme=new_version|is_onsite_check=0|is_gs_filt=0|proxy_ip=0|se_rst=206043|magellan_spu=new_version|ser=1001|bran_ab_test=0|is_waterfall=0|set=3|change_viewtype=true|ma

make selected >  >>>>>> successful end 
switch selected on.
next_page: False


In [12]:
p4p.monitor(group=group, sub_budget_limited=True)

Task Monitor start to run with group="关注词_2"
switch to group: 关注词_2
index: 0 8 > clear band lashes >  >>>>>> successful end 
index: 1 8 > clear band mink eyelashes >  >>>>>> successful end 
index: 2 8 > eyelash extensions private label >  >>>>>> successful end 
index: 3 8 > faux mink eyelashes >  >>>>>> successful end 
index: 4 8 > mink fur eyelashes > 不需要改变价格 >  >>>>>> successful end 
index: 5 8 > natural eyelashes >  >>>>>> successful end 
index: 6 8 > silk eyelashes private label >  >>>>>> successful end 
index: 7 8 > thin band eyelashes > 不需要改变价格 >  >>>>>> successful end 
next_page: False
next_page: False


In [ ]:
p4p.turn_all_off(group=group)

In [30]:
text = 

In [15]:
re.sub(': *"[]",', '', text)

'"needCompareOfferAttr": "" === "true", '

In [32]:
text

'"" === "true"'